In [3]:
import pandas as pd

In [4]:
import os
os.environ["RD_LIB_CONFIG_PATH"] = "../Configuration"

In [6]:
import refinitiv.data as rd

In [7]:
rd.open_session()

An error occurred while requesting URL('http://localhost:9001/api/status').
	ReadTimeout('timed out')


<refinitiv.data.session.Definition object at 0x169b5e2cd00 {name='workspace'}>

In [8]:
df_companies = pd.read_csv('Refinitiv_ESG_Universe.csv', delimiter=';')

df_companies.head()

,ISIN,Name
0,DK0010244508,A P MOLLER MAERSK B
1,DE000A1TNNN5,A S CREATION TAPETEN
2,CA0002551095,A&W REVENUE RYLT.INC.FD. UTS.
3,CNE100002RY5,A-LIVING SERVICES 'H'
4,US00181T1079,A-MARK PRECIOUS METALS


In [9]:
df_companies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9960 entries, 0 to 9959
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ISIN    9960 non-null   object
 1   Name    9960 non-null   object
dtypes: object(2)
memory usage: 155.8+ KB


In [14]:
# example, also works with ISIN instead of BASF Ticker
df = rd.get_data(
    universe=["BASFn.DE", "IBM.N"],
    fields=["TR.HeadquartersCountry", "TR.HeadquartersRegion", "TR.NAICSNationalIndustry", "TR.NAICSIndustryGroup", 
            "TR.NAICSSector", "TR.GICSSubIndustry", "TR.GICSIndustry", "TR.ICBIndustry", "TR.ICBSector", "TR.GICSIndustryGroup", "TR.SICIndustryGroup"
            "TR.GICSSector", "TR.SICIndustry", "TR.TRBCIndustry", "TR.TRBCIndustryGroup", "TR.TRBCBusinessSector", "TR.TRBCEconomicSector",
            "TR.TSE33SectorNameMain", "TR.NAICSSubsector"]
)

df

,Instrument,Revenue,Employees - Average,Country of Headquarters,Region of Headquarters,NAICS National Industry Name,NAICS Industry Group Name,NAICS Sector Name,GICS Sub-Industry Name,GICS Industry Name,ICB Industry name,ICB Sector name,GICS Industry Group Name,SIC Industry Name,TRBC Industry Name,TRBC Industry Group Name,TRBC Business Sector Name,TRBC Economic Sector Name,TSE33 Main sector name,NAICS Subsector Name
0,BASFn.DE,87327000000,111023,Germany,Europe,Paint and Coating Manufacturing,"Paint, Coating, and Adhesive Manufacturing",Manufacturing,Diversified Chemicals,Chemicals,Basic Materials,Chemicals,Materials,,Diversified Chemicals,Chemicals,Chemicals,Basic Materials,,Chemical Manufacturing
1,IBM.N,60530000000,<NA>,United States of America,Americas,Computer Systems Design Services,Computer Systems Design and Related Services,"Professional, Scientific, and Technical Services",IT Consulting & Other Services,IT Services,Technology,Software & Computer Services,Software & Services,,IT Services & Consulting,Software & IT Services,Software & IT Services,Technology,,"Professional, Scientific, and Technical Services"


In [10]:
# Stuff that doesn't change with time (Sector/Industry/Country/Region)
# testing the loop
test_frame = df_companies.iloc[:15, :]

# loop to collect data for all available companies, stored in a dictionary with a df for each isin
comp_data_dict = {}

for isin in test_frame["ISIN"]:
    df = rd.get_data(
        universe=isin,
        fields=["TR.HeadquartersCountry", "TR.HeadquartersRegion", "TR.NAICSNationalIndustry", "TR.NAICSIndustryGroup", 
            "TR.NAICSSector", "TR.GICSSubIndustry", "TR.GICSIndustry", "TR.ICBIndustry", "TR.ICBSector", "TR.GICSIndustryGroup", "TR.SICIndustryGroup"
            "TR.GICSSector", "TR.SICIndustry", "TR.TRBCIndustry", "TR.TRBCIndustryGroup", "TR.TRBCBusinessSector", "TR.TRBCEconomicSector",
            "TR.TSE33SectorNameMain", "TR.NAICSSubsector"]
    )

    comp_data_dict[isin] = df

comp_dfs = []
for key, df in comp_data_dict.items():
    df['key'] = key  # Add a new column containing the key
    comp_dfs.append(df)

# Concatenate the dataframes into a single one
comp_info_df_test = pd.concat(comp_dfs)

# save the new dataframe
comp_info_df_test.to_csv("comp_info_df_test.csv")

In [12]:
comp_info_df_test.head()

,Instrument,Country of Headquarters,Region of Headquarters,NAICS National Industry Name,NAICS Industry Group Name,NAICS Sector Name,GICS Sub-Industry Name,GICS Industry Name,ICB Industry name,ICB Sector name,GICS Industry Group Name,SIC Industry Name,TRBC Industry Name,TRBC Industry Group Name,TRBC Business Sector Name,TRBC Economic Sector Name,TSE33 Main sector name,NAICS Subsector Name,key
0,DK0010244508,Denmark,Europe,Deep Sea Freight Transportation,"Deep Sea, Coastal, and Great Lakes Water Trans...",Transportation and Warehousing,Marine,Marine,Industrials,Industrial Transportation,Transportation,,Marine Freight & Logistics,Freight & Logistics Services,Transportation,Industrials,,Water Transportation,DK0010244508
0,DE000A1TNNN5,Germany,Europe,Paper Bag and Coated and Treated Paper Manufac...,Converted Paper Product Manufacturing,Manufacturing,Home Furnishings,Household Durables,Consumer Discretionary,Household Goods and Home Construction,Consumer Durables & Apparel,,Home Furnishings,Household Goods,Cyclical Consumer Products,Consumer Cyclicals,,Paper Manufacturing,DE000A1TNNN5
0,CA0002551095,Canada,Americas,Full-Service Restaurants,Restaurants and Other Eating Places,Accommodation and Food Services,Restaurants,"Hotels, Restaurants & Leisure",Consumer Discretionary,Travel and Leisure,Consumer Services,,Restaurants & Bars,Hotels & Entertainment Services,Cyclical Consumer Services,Consumer Cyclicals,,Food Services and Drinking Places,CA0002551095
0,CNE100002RY5,China,Asia,Residential Property Managers,Activities Related to Real Estate,Real Estate and Rental and Leasing,Real Estate Operating Companies,Real Estate Management & Development,Real Estate,Real Estate Investment and Services Development,Real Estate,,Real Estate Services,Real Estate Operations,Real Estate,Real Estate,,Real Estate,CNE100002RY5
0,US00181T1079,United States of America,Americas,"Jewelry, Watch, Precious Stone, and Precious M...",Miscellaneous Durable Goods Merchant Wholesalers,Wholesale Trade,Specialized Finance,Diversified Financial Services,Consumer Discretionary,Retailers,Diversified Financials,"Jewelry, Watches, Precious Stones, and Preciou...",Non-Gold Precious Metals & Minerals,Metals & Mining,Mineral Resources,Basic Materials,,"Merchant Wholesalers, Durable Goods",US00181T1079


In [ ]:
# Stuff that doesn't change with time (Sector/Industry/Country/Region)

# loop to collect data for all available companies, stored in a dictionary with a df for each isin
comp_data_dict = {}

for isin in df_companies["ISIN"]:
    
    df = rd.get_data(
        universe=isin,
        fields=["TR.HeadquartersCountry", "TR.HeadquartersRegion", "TR.NAICSNationalIndustry", "TR.NAICSIndustryGroup", 
            "TR.NAICSSector", "TR.GICSSubIndustry", "TR.GICSIndustry", "TR.ICBIndustry", "TR.ICBSector", "TR.GICSIndustryGroup", "TR.SICIndustryGroup"
            "TR.GICSSector", "TR.SICIndustry", "TR.TRBCIndustry", "TR.TRBCIndustryGroup", "TR.TRBCBusinessSector", "TR.TRBCEconomicSector",
            "TR.TSE33SectorNameMain", "TR.NAICSSubsector"]
    )

    comp_data_dict[isin] = df

comp_dfs = []
for key, df in comp_data_dict.items():
    df['key'] = key  # Add a new column containing the key
    comp_dfs.append(df)

# Concatenate the dataframes into a single one
comp_info_df = pd.concat(comp_dfs)

# save the new dataframe
comp_info_df.to_csv("comp_info_df.csv")

In [16]:
# testing the loop
test_frame = df_companies.iloc[:15, :]

# loop to collect data for all available companies, stored in a dictionary with a df for each isin
ESG_data_dict = {}

for isin in test_frame["ISIN"]:

    df = rd.get_history(
        universe=isin,
        fields=["TR.Revenue", "TR.F.EmpAvg"],
        interval="1Y",
        start="2000-01-01",
        end="2023-01-01",
    )
    
    df.index = pd.to_datetime(df.index)
    df = df.groupby(df.index.year).first()

    ESG_data_dict[isin] = df

# Assume `dfs_dict` is the dictionary of dataframes
dfs = []
for key, df in ESG_data_dict.items():
    df['key'] = key  # Add a new column containing the key
    dfs.append(df)

# Concatenate the dataframes into a single one
result = pd.concat(dfs)

# save the new dataframe
result.to_csv('comp_info_test.csv')

result.head()

,Revenue,Employees - Average,Country of Headquarters,Region of Headquarters,ICB Industry name,ICB Sector name,key
1999,81346688,<NA>,,,,,DK0010244508
2000,71846460,<NA>,,,,,DK0010244508
2002,79460058,<NA>,,,,,DK0010244508
2003,23901605672,<NA>,,,,,DK0010244508
2004,26648634320,<NA>,,,,,DK0010244508


In [12]:
# loop to collect data for all available companies, stored in a dictionary with a df for each isin
ESG_data_dict = {}

for isin in df_companies["ISIN"]:

    df = rd.get_history(
        universe=isin,
        fields=["TR.TotalReturn", "TR.TRESGScore", "TR.TRESGCScore",
                "TR.TRESGCControversiesScore", "TR.SocialPillarScore",
                "TR.GovernancePillarScore", "TR.EnvironmentPillarScore"],
        interval="1Y",
        start="2000-01-01",
        end="2023-01-01",
    )
    
    df.index = pd.to_datetime(df.index)
    df = df.groupby(df.index.year).first()

    ESG_data_dict[isin] = df

esg_dfs = []
for key, df in ESG_data_dict.items():
    df['key'] = key  # Add a new column containing the key
    esg_dfs.append(df)

# Concatenate the dataframes into a single one
tr_esg_df = pd.concat(esg_dfs)

# save the new dataframe
tr_esg_df.to_csv('tr_esg_df.csv')

In [13]:
tr_esg_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 224852 entries, 2000 to 2022
Data columns (total 8 columns):
 #   Column                      Non-Null Count   Dtype 
---  ------                      --------------   ----- 
 0   Total Return                163645 non-null  object
 1   ESG Score                   79945 non-null   object
 2   ESG Combined Score          79945 non-null   object
 3   ESG Controversies Score     79936 non-null   object
 4   Social Pillar Score         79936 non-null   object
 5   Governance Pillar Score     79945 non-null   object
 6   Environmental Pillar Score  79936 non-null   object
 7   key                         224852 non-null  object
dtypes: object(8)
memory usage: 15.4+ MB


In [3]:
tr_esg_df_cleaned = df.dropna().rename(columns={"Unnamed: 0": "Year"})

tr_esg_df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 78518 entries, 2 to 224850
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Year                        78518 non-null  int64  
 1   Total Return                78518 non-null  float64
 2   ESG Score                   78518 non-null  float64
 3   ESG Combined Score          78518 non-null  float64
 4   ESG Controversies Score     78518 non-null  float64
 5   Social Pillar Score         78518 non-null  float64
 6   Governance Pillar Score     78518 non-null  float64
 7   Environmental Pillar Score  78518 non-null  float64
 8   key                         78518 non-null  object 
dtypes: float64(7), int64(1), object(1)
memory usage: 6.0+ MB


In [4]:
tr_esg_df_cleaned.head()

,Year,Total Return,ESG Score,ESG Combined Score,ESG Controversies Score,Social Pillar Score,Governance Pillar Score,Environmental Pillar Score,key
2,2002,-4.703688,10.732601,10.732601,100.0,16.469428,16.153846,0.000000,DK0010244508
3,2003,78.725725,16.933543,16.933543,100.0,19.033531,34.523810,0.000000,DK0010244508
4,2004,6.899061,17.855846,17.855846,100.0,20.465337,26.862745,7.523148,DK0010244508
5,2005,45.463359,16.395483,16.395483,100.0,20.152244,17.166667,11.683007,DK0010244508
6,2006,-17.550102,18.732474,18.732474,60.0,20.633013,25.500000,11.033951,DK0010244508


In [5]:
tr_esg_df_cleaned.to_csv("tr_esg_df_cleaned.csv")

In [ ]:
rd.close_session()